In [4]:
import os
from keras.preprocessing import image
import cv2

In [5]:
categories = ['without_mask','with_mask']

In [6]:
data = []

for category in categories:
    path = os.path.join('Covid-19-PIS.v2i.folder\\dataset',category)
    label=categories.index(category)

    for file in os.listdir(path):
        img_path = os.path.join(path,file)
        img=cv2.imread(img_path)
        img=cv2.resize(img,(224,224)) 
        
        data.append([img,label])



In [7]:
import random
random.shuffle(data)

In [8]:
x,y=[],[] 

for feature,label in data:
    x.append(feature)
    y.append(label)

In [9]:
import numpy as np
x=np.array(x)
y=np.array(y)

In [10]:
x=x/255

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [12]:
from keras.applications.vgg16 import VGG16

vgg=VGG16()

In [13]:
from keras import Sequential
model=Sequential()


In [14]:
for layer in vgg.layers[:-1]:
    model.add(layer)
    

In [15]:
for layer in model.layers:
    layer.trainable=False


In [16]:
from keras.layers import Dense

model.add(Dense(1,activation='sigmoid'))

In [17]:
model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy'])

In [18]:
model.fit(X_train,y_train,epochs=4,validation_data=(X_test,y_test))

Epoch 1/4
32/32 ━━━━━━━━━━━━━━━━━━━━ 127s 4s/step - accuracy: 0.5578 - loss: 0.7535 - val_accuracy: 0.7617 - val_loss: 0.4800
Epoch 2/4
32/32 ━━━━━━━━━━━━━━━━━━━━ 113s 4s/step - accuracy: 0.8441 - loss: 0.4240 - val_accuracy: 0.9180 - val_loss: 0.3101
Epoch 3/4
32/32 ━━━━━━━━━━━━━━━━━━━━ 110s 3s/step - accuracy: 0.9085 - loss: 0.3024 - val_accuracy: 0.9219 - val_loss: 0.2607
Epoch 4/4
32/32 ━━━━━━━━━━━━━━━━━━━━ 110s 3s/step - accuracy: 0.9254 - loss: 0.2435 - val_accuracy: 0.9297 - val_loss: 0.2257


In [19]:
cap = cv2.VideoCapture(0)

In [27]:
while True:
    ret,frame=cap.read()
    cv2.imshow('window',frame)
    frame=cv2.resize(frame,(224,224))

    y_pred=detect_face_mask(frame)

    if y_pred == 0:
        draw_label(frame,"mask",(30,30),(0,255,0))
    else:
        draw_label(frame," no mask",(30,30),(0,255,0))

    if cv2.waitKey(1) & 0xFF==ord('x'):
        break
    
    if cv2.getWindowProperty('window', cv2.WND_PROP_VISIBLE) < 1:
        break

cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [22]:
def detect_face_mask(img):
    y_pred=model.predict(img.reshape(1,224,224,3))
    return y_pred[0][0]


In [26]:
def draw_label(img,text,pos,bg_color):
    text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    end_x = pos[0] + text_size[0][0]+2
    end_y = pos[0] + text_size[0][0]-2

    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)
    
    


